In [68]:
from io import StringIO
from pathlib import Path
import json
import pandas as pd

In [69]:
def json_to_df(_json):
    f = StringIO(_json)
    return pd.read_json(f)

In [70]:
studies_excel = pd.read_excel("data/chloroform-studies.xlsx")
original_ids = studies_excel["original_id"].tolist()
new_ids = studies_excel["new_id"].tolist()

fn = Path('data/raw/writes.json')
writes = json.loads(fn.read_text())

## Study

In [71]:
# Only write non-created studies
study_df = json_to_df(writes["study"]["study_study"])
study_ids = studies_excel[studies_excel["recreated"] == False]["original_id"].tolist()
study_df = study_df[study_df["reference_ptr_id"].isin(study_ids)]
# Replace old study ids with new study ids
study_df["reference_ptr_id"].replace(original_ids,new_ids,inplace=True)
writes["study"]["study_study"] = study_df.to_json()

## Animal

In [72]:
# Replace experiment study ids with new study ids
experiment_df = json_to_df(writes["animal"]["animal_experiment"])
experiment_df["study_id"].replace(original_ids,new_ids,inplace=True)
writes["animal"]["animal_experiment"] = experiment_df.to_json()

## Risk of bias

In [73]:
# Replace rob study ids with new study ids
rob_df = json_to_df(writes["rob"]["riskofbias_riskofbias"])
rob_df["study_id"].replace(original_ids,new_ids,inplace=True)
writes["rob"]["riskofbias_riskofbias"] = rob_df.to_json()

## Save to file

In [74]:
# Save transformed data
fn = Path('data/transformed/writes.json')
fn.write_text(json.dumps(writes))

1284049